# Protein annotation SARS-CoV-2

This notebook uses Python tools to find all amino acid mutations in SARS-CoV-2 protein sequences.

Links:

* https://www.gisaid.org/
* http://virological.org/t/selection-analysis-of-gisaid-sars-cov-2-data/448/2

In [347]:
import os, math, time, pickle, subprocess
from importlib import reload
from collections import OrderedDict, defaultdict
import numpy as np
import pandas as pd
pd.set_option('display.width', 150)
from Bio import SeqIO,AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pygenefinder import app,tools

In [271]:
reload(tools)
sc2ref = tools.genbank_to_dataframe('NC_045512.2.gb',cds=True).drop_duplicates('locus_tag')
sc2ref[['gene','locus_tag','length','start']]

,gene,locus_tag,length,start
3,ORF1ab,GU280_gp01,7096,265
34,S,GU280_gp02,1273,21562
36,ORF3a,GU280_gp03,275,25392
38,E,GU280_gp04,75,26244
40,M,GU280_gp05,222,26522
42,ORF6,GU280_gp06,61,27201
44,ORF7a,GU280_gp07,121,27393
46,ORF7b,GU280_gp08,43,27755
48,ORF8,GU280_gp09,121,27893
50,N,GU280_gp10,419,28273


## annotate reference to check match with genbank one

In [292]:
reload(app)
sc2,sc2recs = app.run_annotation('NC_045512.fa', kingdom='viruses')
sc2[['gene','product','length','start']]

/home/damien/.config/pygenefinder/prokka/IS.fa
blasting 10 ORFs to IS
blastp -out tempseq_blast.txt -outfmt "6 qseqid sseqid qseq sseq pident qcovs length mismatch gapopen qstart qend sstart send evalue bitscore stitle" -query tempseq.fa -db /home/damien/.config/pygenefinder/prokka/IS.fa -evalue 1e-10 -max_target_seqs 1 -num_threads 4
/home/damien/.config/pygenefinder/prokka/amr.fa
blasting 10 ORFs to amr
blastp -out tempseq_blast.txt -outfmt "6 qseqid sseqid qseq sseq pident qcovs length mismatch gapopen qstart qend sstart send evalue bitscore stitle" -query tempseq.fa -db /home/damien/.config/pygenefinder/prokka/amr.fa -evalue 1e-100 -max_target_seqs 1 -num_threads 4
/home/damien/.config/pygenefinder/prokka/sprot_viruses.fa
blasting 10 ORFs to sprot_viruses
blastp -out tempseq_blast.txt -outfmt "6 qseqid sseqid qseq sseq pident qcovs length mismatch gapopen qstart qend sstart send evalue bitscore stitle" -query tempseq.fa -db /home/damien/.config/pygenefinder/prokka/sprot_viruses.fa 

,gene,product,length,start
0,1a,Replicase polyprotein 1a,4388,317
1,rep,Replicase polyprotein 1ab,2595,13768
2,S,Spike glycoprotein,1273,21563
3,3a,Protein 3a,275,25393
4,M,Membrane protein,222,26523
5,7a,Protein 7a,121,27394
6,N,Nucleoprotein,419,28274
7,NaN,hypothetical protein,43,27756
8,NaN,hypothetical protein,121,27894
9,NaN,hypothetical protein,51,29685


In [362]:
spikeref = sc2[sc2.gene=='S'].iloc[0]
spikeref.translation
refrec = SeqRecord(Seq(spikeref.translation),id='ref')

## Load unique GIS sequences 

In [ ]:
#gistable = pd.read_excel('gisaid_cov2020_acknowledgement_table.xls')

def load_deduplicated_sequences(filename):
    """Load a fasta file of sequences and ignore duplicates"""
    
    newrecs = {}
    unique_seqs=defaultdict(list)
    with open(filename, 'r') as in_handle:
        for rec in SeqIO.parse(in_handle, "fasta"):        
            if rec.seq in unique_seqs:
                continue
            if not rec.id in gisrecs:
                try:
                    id = rec.id.split('|')[1]           
                    newrecs[id] = rec
                    #seqs.append(rec.seq)
                    unique_seqs[str(rec.seq)] = id
                except:
                    pass
    return newrecs   

gisrecs = load_deduplicated_sequences('gisaid_cov2020_sequences.fasta')
print (len(gisrecs))

In [325]:
reload(app)
k = list(gisrecs.keys())[:2000]
annot = app.annotate_files(gisrecs, keys=k, outdir='gisaid_annot')

## Extract protein of interest across all genomes

In [363]:
spike_seqs = app.get_similar_sequences('Spike glycoprotein', annot)
print (len(spike_seqs))

1998


## Collapse to unique protein sequences

In [364]:
unique_seqs, counts = tools.collapse_sequences(spike_seqs, refrec)
print (len(unique_seqs))

117


## Find mutations

In [ ]:
def get_mutations(recs, ref):
    mutations = {}
    positions = []
    for rec in recs:
        aln = tools.clustal_alignment(seqs=[ref, rec])
        #print (aln)
        x = []
        for pos in range(len(aln[0])):
            refaa = aln[0,pos]        
            aa = aln[1,pos]
            if aa != refaa and aa!='-':
                #print (refaa, aln[:,pos], aa)              
                mut = refaa+str(pos+1)+aa
                x.append(mut)
        if len(x)>0:
            mutations[rec.seq] = x
    return mutations

mutations = get_mutations(unique_seqs, refrec)
mutations.values()

In [377]:
muts = {seq:'+'.join(mutations[seq]) for seq in mutations if counts[seq]>1}
freqs = [(muts[seq], counts[seq]) for seq in muts]
freqs
fdf = pd.DataFrame(freqs,columns=['mutation','count']).sort_values('count',ascending=False)

## make a phylogenetic tree from protein sequences

In [358]:
def make_tree(infile, kind='nucl'):
    
    #aln = tools.clustal_alignment(seqs=seqs)
    cmd = 'mafft --retree 1 %s > aligned.fasta' %infile
    print (cmd)
    subprocess.check_output(cmd, shell=True)
    #AlignIO.write(aln, 'aligned.fasta', 'fasta')
    AlignIO.convert("aligned.fasta", "fasta", "aligned.phy", "phylip-relaxed")
    if kind=='nucl':
        cmd = 'mpirun -np 4 phyml-mpi -i aligned.phy -b 12 -m HKY85 -a e'
    else:
        cmd = 'mpirun -np 4 phyml-mpi -i aligned.phy -d aa -b 12 -m WAG -a e'
    tmp = subprocess.check_output(cmd, shell=True)
    return 

In [ ]:
#write dict with mutations to a fasta file
temp={}
for s in muts:
    rec=SeqRecord(s,id=muts[s])
    temp[muts[s]] = rec
SeqIO.write(temp.values(),'temp_mutants.faa','fasta')
make_tree('temp_mutants.faa',kind='prot')

In [ ]:
from Bio import Phylo
tree = Phylo.read("aligned.phy_phyml_tree.txt", "newick")
Phylo.draw_ascii(tree)

In [ ]:
import nglview
w = nglview.show_file("model_spike.pdb")
w.add_representation('licorice', selection=positions, color='green')
w

## Run all proteins using the above methods

In [304]:
print (annot.groupby(['product','gene']).agg({'sequence':np.size}).reset_index())

                           product gene  sequence
0  Envelope small membrane protein    E       146
1                 Membrane protein    M      6406
2        Non-structural protein 3b   3b         4
3                    Nucleoprotein    N      6371
4                       Protein 3a   3a      6387
5                       Protein 7a   7a      6359
6                       Protein 9b   9b       138
7         Replicase polyprotein 1a   1a      6413
8        Replicase polyprotein 1ab  rep      6342
9               Spike glycoprotein    S      6407


In [ ]:
#k = list(gisrecs.keys())[:500]
#annot = app.annotate_files(gisrecs, outdir='gisaid_annot', kingdom='viruses')

names = ['Protein 7a', 'Protein 3a', 'Spike glycoprotein','Membrane protein',
         'Nucleoprotein','Replicase polyprotein 1a','Replicase polyprotein 1ab']
res=[]
mutant_seqs = {}
for protname in names:
    refprot = sc2[sc2['product']==protname].iloc[0]
    refrec = SeqRecord(Seq(refprot.translation),id='ref')
    print (protname)
    annot_seqs = app.get_similar_sequences(protname, annot)
    unique_seqs, counts = tools.collapse_sequences(annot_seqs, refrec)
    print ('%s unique sequences' %len(unique_seqs))
    mutations = get_mutations(unique_seqs, refrec)
    #convert mutations to string and count the frequency
    muts = {seq:'+'.join(mutations[seq]) for seq in mutations}
    #save the sequences for later use
    mutant_seqs[protname] = muts
    freqs = [(muts[seq], counts[seq]) for seq in muts]    
    #convert freq table to dataframe
    fdf = pd.DataFrame(freqs,columns=['mutation','count']).sort_values('count',ascending=False)
    fdf['protein'] = protname
    print (fdf[:10])
    res.append(fdf)

res = pd.concat(res).sort_values('count',ascending=False)
res.to_csv('sarscov2_mutations.csv')

In [ ]:
#fdf = pd.DataFrame(freqs,columns=['mutation','count']).sort_values('count',ascending=False)
res[:50].plot(x='mutation',kind='bar',width=.9,figsize=(10,4))
import pylab as plt
plt.tight_layout()
plt.savefig('sarscov2_mut_freq.png',dpi=100)
res[:20]